<a href="https://colab.research.google.com/github/amirpaia/blenderbot/blob/main/blender_90m_with_double_sized_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.Installing prerequisites

In [42]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue May 31 13:08:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    34W /  70W |  14214MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [43]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
mydrive_path = '/content/drive/MyDrive/colabs/blender-models/'
# !pip uninstall -q parlai
!pip install -q parlai
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 1. Preparing Data

## Genreal Functions

In [46]:
def transfer_list_of_turns_to_dialog(d):
    if len(d)%2 !=0: d = d[:-1]
    t = ""
    for i in range(0,len(d),2):
        u1 = d[i]
        u2 = d[i+1]

        if (i+2) != len(d):
            t += "text:"+u1+"\t"+"labels:"+u2+"\n"
        else:
            t += "text:"+u1+"\t"+"labels:"+u2+"\t"+"episode_done:True"+"\n"
    return t

def transfer_list_of_pairs_to_dialog(d):
  t = ""
  for i, text_label_pair in enumerate(d):
    u1 = text_label_pair[0]
    u2 = text_label_pair[1]

    if i != (len(d) - 1):
      t += "text:"+u1+"\t"+"labels:"+u2+"\n"
    else:
      t += "text:"+u1+"\t"+"labels:"+u2+"\t"+"episode_done:True"+"\n"

  return t

def convert_parlai_format_to_list_of_turns(lines):
    result = []
    for line in lines:
        text_label = line.split("\t")
        result.append(text_label[0].replace("text:", ""))
        result.append(text_label[1].replace("labels:", "").replace("\n",""))
    return result

t = ['hello','how are you','good','bye','test']
print(transfer_list_of_turns_to_dialog(t))

t = [['hello','how are you'],['good','bye']]
print(transfer_list_of_pairs_to_dialog(t))

text:hello	labels:how are you
text:good	labels:bye	episode_done:True

text:hello	labels:how are you
text:good	labels:bye	episode_done:True



In [ ]:
# def make_dataset_double_sided(lines, path, filename): 
#     # train[:10]
#     all_dialogs_parlai_format = []
#     dialog = []
#     for line in lines:
#         dialog.append(line)
#         if 'episode_done:True' in line:
#             turns = convert_parlai_format_to_list_of_turns(dialog)
#             first_parlai_dialog = transfer_list_of_turns_to_dialog(turns)
#             second_parlai_dialog = transfer_list_of_turns_to_dialog(turns[1:])

#             all_dialogs_parlai_format.append(first_parlai_dialog)
#             all_dialogs_parlai_format.append(second_parlai_dialog)

#             dialog = []
#             # break

#     print(len(all_dialogs_parlai_format))

#     with open(f"{path}{filename}", "w") as f:
#         f.writelines(all_dialogs_parlai_format[:10])

# !mkdir /content/dataset_french_bst/
# make_dataset_double_sided(lines,"/content/dataset_french_bst/", "train.txt")



# # import os.path
# # from os import path

## XPersona

In [47]:
!git clone https://github.com/HLTCHKUST/Xpersona.git

import json
import pandas as pd

with open('Xpersona/dataset/Fr_persona_train_corrected.json','r') as f:
   train_data = json.load(f)

dialogs_train = pd.DataFrame(train_data)['dialogue'].tolist()

with open('Xpersona/dataset/Fr_persona_split_valid_human_annotated.json','r') as f:
   valid_data = json.load(f)

dialogs_valid = pd.DataFrame(valid_data)['dialogue'].tolist()

with open('Xpersona/dataset/Fr_persona_split_test_human_annotated.json','r') as f:
   test_data = json.load(f)

dialogs_test = pd.DataFrame(test_data)['dialogue'].tolist()

# dialogs

fatal: destination path 'Xpersona' already exists and is not an empty directory.


In [48]:
data_path = "/content/dataset_french_xpersona/"
!rm -R $data_path
!mkdir $data_path

def convert_xpersona_dialogs_to_parlai_format_file_and_double_sided(dialogs, filename):
    all_dialogs_parlai_format = []
    import numpy as np
    for d in dialogs:
        turns = np.reshape(d, (-1)).tolist()
        all_dialogs_parlai_format.append(transfer_list_of_turns_to_dialog(turns))
        all_dialogs_parlai_format.append(transfer_list_of_turns_to_dialog(turns[1:]))

    print(len(all_dialogs_parlai_format))

    with open(f"{data_path}{filename}","w") as f:
        f.writelines(all_dialogs_parlai_format)

convert_xpersona_dialogs_to_parlai_format_file_and_double_sided(dialogs_train, "train.txt")
convert_xpersona_dialogs_to_parlai_format_file_and_double_sided(dialogs_valid, "valid.txt")
convert_xpersona_dialogs_to_parlai_format_file_and_double_sided(dialogs_test, "test.txt")

33756
496
498


In [49]:
# data_path = "/content/dataset_french_xpersona/"
# !rm -R $data_path
# !mkdir $data_path

# #region Training
# data_train = ""
# for d in dialogs_train:
#   data_train += transfer_list_of_pairs_to_dialog(d)

# file_train = open(f"{data_path}train.txt","w")
# print("Training Set:", file_train.write(data_train))
# #endregion 
    
# #region Validation
# data_valid = ""
# for d in dialogs_valid:
#   data_valid += transfer_list_of_pairs_to_dialog(d)

# file_valid = open(f"{data_path}valid.txt","w")
# print("Validation Set:", file_valid.write(data_valid))
# #endregion

# #region Test
# data_test = ""
# for d in dialogs_test:
#   data_test += transfer_list_of_pairs_to_dialog(d)

# file_test = open(f"{data_path}test.txt","w")
# print("Test Set:", file_test.write(data_test))
# #endregion 

# # print(len(data_valid))

## ED

In [50]:
googledrive_data_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_empathetic_dialogues/"
data_path = "/content/dataset_french_ed/"
!rm -R $data_path
!mkdir $data_path
!cp -rv $googledrive_data_path* $data_path

'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_empathetic_dialogues/test.txt' -> '/content/dataset_french_ed/test.txt'
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_empathetic_dialogues/train.txt' -> '/content/dataset_french_ed/train.txt'
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_empathetic_dialogues/valid.txt' -> '/content/dataset_french_ed/valid.txt'


In [ ]:
# with open(f"{googledrive_data_path}train.txt") as f:
#     train = f.readlines()

# with open(f"{googledrive_data_path}valid.txt") as f:
#     valid = f.readlines()

# with open(f"{googledrive_data_path}test.txt") as f:
#     test = f.readlines()


# print(len(train))
# a, b, c = sum([1 for a in train if 'episode_done:True' in a]), sum([1 for a in valid if 'episode_done:True' in a]), sum([1 for a in test if 'episode_done:True' in a])
# print(a,b,c,a+b+c)


64636
39057 5537 5093 49687


## BST

In [51]:
googledrive_data_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/"
data_path = "/content/dataset_french_bst/"
!rm -R $data_path
!mkdir $data_path
!cp -rv $googledrive_data_path* $data_path

'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/test.txt' -> '/content/dataset_french_bst/test.txt'
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/train.txt' -> '/content/dataset_french_bst/train.txt'
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/valid.txt' -> '/content/dataset_french_bst/valid.txt'


In [52]:
googledrive_data_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/"
with open(f"{googledrive_data_path}train.txt") as f:
    train = f.readlines()

with open(f"{googledrive_data_path}valid.txt") as f:
    valid = f.readlines()

with open(f"{googledrive_data_path}test.txt") as f:
    test = f.readlines()


print(len(train))
a, b, c = sum([1 for a in train if 'episode_done:True' in a]), sum([1 for a in valid if 'episode_done:True' in a]), sum([1 for a in test if 'episode_done:True' in a])
print(a, b, c, a+b+c)




59700
9638 2018 1958 13614


# 2. Creating new Tasks

In [53]:
#region XPersona
task_path = "/usr/local/lib/python3.7/dist-packages/parlai/tasks/"

!rm -R $task_path'french_xpersona/'
!mkdir $task_path'french_xpersona'
!cp -ruv /content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_xpersona/* $task_path'french_xpersona/'
#endregion


#region ED
task_path = "/usr/local/lib/python3.7/dist-packages/parlai/tasks/"

!rm -R $task_path'french_empathetic_dialogues/'
!mkdir $task_path'french_empathetic_dialogues'
!cp -ruv /content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_empathetic_dialogues/* $task_path'french_empathetic_dialogues/'
#endregion


#region BST
task_path = "/usr/local/lib/python3.7/dist-packages/parlai/tasks/"

!rm -R $task_path'french_blended_skill_talk/'
!mkdir $task_path'french_blended_skill_talk'
!cp -ruv /content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_blended_skill_talk/* $task_path'french_blended_skill_talk/'
#endregion

'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_xpersona/agents.py' -> '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_xpersona/agents.py'
'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_xpersona/build.py' -> '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_xpersona/build.py'
'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_xpersona/__init__.py' -> '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_xpersona/__init__.py'
'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_xpersona/LICENSE_DOCUMENTATION' -> '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_xpersona/LICENSE_DOCUMENTATION'
'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_xpersona/README.md' -> '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_xpersona/README.md'
'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_xpersona/test' -> '/usr/local/lib/python3.7/dist-packages

# 3. Finetuning

In [69]:
finetuned_model_path = f'{mydrive_path}finetuned-multitask-400m-double-sided'
init_model = 'zoo:blender/blender_400Mdistill/model'
dict_file  = 'zoo:blender/blender_400Mdistill/model.dict'
finetuned_model_path

'/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-double-sided'

In [ ]:
# !mkdir copied_dataset_french_bst
# !cp dataset_french_bst/* copied_dataset_french_bst
# !mv copied_dataset_french_bst/test.txt copied_dataset_french_bst/_test.txt
# !mv copied_dataset_french_bst/train.txt copied_dataset_french_bst/_train.txt
# !mv copied_dataset_french_bst/valid.txt copied_dataset_french_bst/_valid.txt

In [ ]:
# 90M settings
# !rm -rf $finetuned_model_path
!mkdir -p $finetuned_model_path


from parlai.scripts.train_model import TrainModel

TrainModel.main(
    # task
    task= "french_blended_skill_talk,french_xpersona,french_empathetic_dialogues",
    multitask_weights= "1,3,3",

    # task='fromfile:parlaiformat', 
    # fromfile_datapath='copied_dataset_french_bst/',
    # fromfile_datatype_extension=True,

    model='transformer/generator',
    model_file= f'{finetuned_model_path}/model',
    
    # initialize with a pretrained model
    init_model= init_model,
    dict_file=dict_file,
    
    # arguments we get from the pretrained model.
    # Unfortunately, these must be looked up separately for each model.
    n_heads=16, n_layers=8, n_positions=512, text_truncate=512,
    label_truncate=128, ffn_size=2048, embedding_size=512,
    activation='gelu', variant='xlm',
    dict_lower=True, dict_tokenizer='bpe',
    
    # depend on your gpu. 
    validation_every_n_epochs=0.25,
    num_epochs = 5,
    log_every_n_secs= 60,
    verbose = True,
    batchsize= 8, 
    fp16= True, fp16_impl= "mem_efficient",
    
    # arguments we get from the pretrained model.
    
    # speeds up validation
    skip_generation=True,
    vp= 10,
    validation_metric= "ppl", #vmt = "ppl"
    validation_metric_mode= "min", # vmm= "min"
    
    # helps us cram more examples into our gpu at a time
    dynamic_batching='full',

    
    # some training arguments, specific to this fine-tuning
    lr=1e-5, optimizer='adam',
    attention_dropout= 0.0, 
    model_parallel= False,
    warmup_updates=100,

    # customized parameters
    # inference= "beam"
    # beam_min_length= 20,
    # beam_block_ngram= 3,
    # beam_context_block_ngram= 3,
    # beam_size= 10,
)

In [82]:
# mydrive_path = '/content/finetuned-multitask-400m-double-sided-2epochs'
# mydrive_path = '/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-double-sided/'

In [83]:
# 400M settings
# !rm -rf $finetuned_model_path
!mkdir -p $finetuned_model_path


from parlai.scripts.train_model import TrainModel

TrainModel.main(
    # task
    task= "french_blended_skill_talk,french_xpersona,french_empathetic_dialogues",
    multitask_weights= "1,3,3",

    # task='fromfile:parlaiformat', 
    # fromfile_datapath='copied_dataset_french_bst/',
    # fromfile_datatype_extension=True,

    model='transformer/generator',
    model_file= f'{finetuned_model_path}/model',
    
    # initialize with a pretrained model
    init_model= init_model,
    dict_file=dict_file,
    
    # depend on your gpu
    validation_every_n_epochs=0.25, # veps= 0.25, 
    num_epochs = 5,
    log_every_n_secs= 60,
    verbose = True,
    attention_dropout= 0.0, 
    batchsize= 16, 
    fp16= True, fp16_impl= "mem_efficient",
    # save_after_valid= True,

    # arguments we get from the pretrained model. "from recipes page for 2.7B model" 
    embedding_size= 1280, ffn_size= 5120,
    variant= "prelayernorm",
    n_heads= 32, n_positions= 128, 
    n_encoder_layers= 2, n_decoder_layers= 12,

    label_truncate= 128, text_truncate= 128, truncate= 128,
    activation= "gelu",
    history_add_global_end_token= "end", 
    delimiter= '  ', 
    dict_tokenizer= "bytelevelbpe",
    dropout= 0.1,
    
    # some training arguments, specific to this fine-tuning
    lr= 7e-06, lr_scheduler= "reduceonplateau", lr_scheduler_patience= 3,
    optimizer= "mem_eff_adam",
    relu_dropout= 0.0, 
    model_parallel= False,
    warmup_updates= 100,
    update_freq= 2,
    gradient_clip= 0.1, 

    # speeds up validation
    skip_generation= True,
    vp= 10,
    validation_metric= "ppl", #vmt = "ppl"
    validation_metric_mode= "min", # vmm= "min"

    # customized parameters
    # inference = 'topk', 
    # temperature = 0.7, 
    # topk=30, 
    # beam_length_penalty=1.03
)

16:13:43 | building dictionary first...
16:13:43 | Overriding opt["multitask_weights"] to (1.0, 3.0, 3.0) (previously: [1.0, 3.0, 3.0])
16:13:43 | Overriding opt["model_file"] to /content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-double-sided/model (previously: /content/finetuned-multitask-400m-double-sided/model)
16:13:43 | Overriding opt["init_model"] to zoo:blender/blender_400Mdistill/model (previously: /content/finetuned-multitask-400m-double-sided/model.checkpoint)
16:13:43 | Overriding opt["dict_file"] to /usr/local/lib/python3.7/dist-packages/data/models/blender/blender_400Mdistill/model.dict (previously: /content/finetuned-multitask-400m-double-sided/model.checkpoint.dict)
16:13:43 | your model is being loaded with opts that do not exist in the model you are initializing the weights with: verbose: True,download_path: None,datapath: /usr/local/lib/python3.7/dist-packages/data,load_from_checkpoint: True,interactive_mode: False
16:13:43 | Using CUDA
16:13:43 | l

KeyboardInterrupt: ignored

In [84]:
# !cp -rv /content/finetuned-multitask-400m-double-sided-2epochs/* /content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-double-sided/
# !ls -lah /content/finetuned-multitask-400m-double-sided-2epochs/
# !ls -lah /content/finetuned-multitask-400m-double-sided/

'/content/finetuned-multitask-400m-double-sided-2epochs/model' -> '/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-double-sided/model'
'/content/finetuned-multitask-400m-double-sided-2epochs/model.checkpoint' -> '/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-double-sided/model.checkpoint'
'/content/finetuned-multitask-400m-double-sided-2epochs/model.checkpoint.dict' -> '/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-double-sided/model.checkpoint.dict'
'/content/finetuned-multitask-400m-double-sided-2epochs/model.checkpoint.dict-merges.txt' -> '/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-double-sided/model.checkpoint.dict-merges.txt'
'/content/finetuned-multitask-400m-double-sided-2epochs/model.checkpoint.dict.opt' -> '/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-double-sided/model.checkpoint.dict.opt'
'/content/finetuned-multitask-400m-double-sided-2epochs/model.check

# 3. Display Model

In [ ]:
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='french_blended_skill_talk',
    model_file= f'{finetuned_model_path}/model',
    num_examples=20,
    skip_generation=False,
)

15:06:36 | Overriding opt["task"] to french_blended_skill_talk (previously: french_blended_skill_talk,french_xpersona,french_empathetic_dialogues)
15:06:36 | Overriding opt["skip_generation"] to False (previously: True)
15:06:36 | Using CUDA
15:06:36 | loading dictionary from /content/drive/MyDrive/colabs/blender-models/finetuned-multitask-90m-double-sided/model.dict
15:06:36 | num words = 54944
15:06:38 | Total parameters: 87,508,992 (86,984,704 trainable)
15:06:38 | Loading existing model params from /content/drive/MyDrive/colabs/blender-models/finetuned-multitask-90m-double-sided/model
15:06:42 | creating task(s): french_blended_skill_talk
15:06:42 | Loading ParlAI text data: /content/dataset_french_bst/valid.txt
15:06:42 | Opt:
15:06:42 |     activation: gelu
15:06:42 |     adafactor_eps: '[1e-30, 0.001]'
15:06:42 |     adam_eps: 1e-08
15:06:42 |     add_p1_after_newln: False
15:06:42 |     aggregate_micro: False
15:06:42 |     allow_missing_init_opts: False
15:06:42 |     attentio

In [ ]:
# from parlai.scripts.display_data import DisplayData
# DisplayData.main(task='empathetic_dialogues', num_examples=10)

from parlai.scripts.display_data import DisplayData
DisplayData.main(
    task='fromfile:parlaiformat',
    fromfile_datapath='copied_dataset_french_bst/',
    fromfile_datatype_extension=True,
    # model_file= f'{finetuned_model_path}/model',
    # dict_file= dict_file,

    num_examples=20,
    # skip_generation=False,

    # beam_min_length= 20,
    # beam_block_ngram= 3,
    # beam_context_block_ngram= 3,
    # beam_size= 10,

    # inference= "beam"
)

12:46:14 | Opt:
12:46:14 |     allow_missing_init_opts: False
12:46:14 |     batchsize: 1
12:46:14 |     datapath: /usr/local/lib/python3.7/dist-packages/data
12:46:14 |     datatype: train:ordered
12:46:14 |     dict_class: None
12:46:14 |     display_add_fields: 
12:46:14 |     download_path: None
12:46:14 |     dynamic_batching: None
12:46:14 |     fromfile_datapath: copied_dataset_french_bst/
12:46:14 |     fromfile_datatype_extension: True
12:46:14 |     hide_labels: False
12:46:14 |     ignore_agent_reply: True
12:46:14 |     image_cropsize: 224
12:46:14 |     image_mode: raw
12:46:14 |     image_size: 256
12:46:14 |     init_model: None
12:46:14 |     init_opt: None
12:46:14 |     is_debug: False
12:46:14 |     loglevel: info
12:46:14 |     max_display_len: 1000
12:46:14 |     model: None
12:46:14 |     model_file: None
12:46:14 |     multitask_weights: [1]
12:46:14 |     mutators: None
12:46:14 |     num_examples: 20
12:46:14 |     override: "{'task': 'fromfile:parlaiformat', '